In [ ]:
Komentarz:
    To możliwe, że Naive Bayes radził sobie lepiej :) 
        https://stackoverflow.com/questions/35360081/naive-bayes-vs-svm-for-classifying-text-data
            
Ogólnie - do przemyślenia, może da radę lepiej...

In [1]:
import data_helpers
import os
import pandas as pd

load_main = True

main_cats = ['ActionName', 'Capability']
dataframes = {cat : {} for cat in main_cats} if load_main else {}

for cat in main_cats:
    path = os.getcwd() + "/Dataframes/" + cat + "/Processed/"
    ls = os.listdir(path)
    for file in ls:
        if (load_main and cat in file) or (not load_main and cat not in file):
            df = pd.read_csv(path + file, encoding='utf-8')

            df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
            df.drop(["a"], axis=1, inplace=True)

            df.rename({"Unnamed: 0.1":"a"}, axis="columns", inplace=True)
            df.drop(["a"], axis=1, inplace=True)

            df_type = file.split('-')[1]
            if load_main:
                dataframes[cat][df_type] = df
            else:
                sub_cat = file.split('-')[0]
                if sub_cat not in dataframes.keys():
                    dataframes[sub_cat] = {}
                dataframes[sub_cat][df_type] = df

if not load_main:
    main_cats = list(dataframes.keys())
main_cats

['ActionName', 'Capability']

In [6]:
SVC??

In [ ]:
ONE vs MANY VERSION FIRST!

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import classification_report
import numpy as np
    
from clf_helpers import do_the_pipeline

def dummy(doc):
    return doc
    
pipeline = Pipeline([      
    ('vectorizer',
#      TfidfVectorizer(
     CountVectorizer(
         preprocessor=dummy, 
         ngram_range=(1, 2),
         analyzer='word',
         binary=False,
#          max_features=2000
     )),
    ('clf', SVC(
        C=100.0, 
        kernel='rbf',
        gamma = 0.001, 
        class_weight = 'balanced'
    )),
])

do_the_pipeline(pipeline, main_cats, dataframes, accuracy=True, report=True, top_features=False, neigh=False)


>>> ActionName:

# RELATIONS
Vectorizer: W słowniku znajduje się 16276 różnych słów
- Zbiór treningowy:
	ACCURACY: 98.3572%
              precision    recall  f1-score   support

  ActionName       0.96      0.99      0.98      1154
NoActionName       1.00      0.98      0.99      2194

   micro avg       0.98      0.98      0.98      3348
   macro avg       0.98      0.99      0.98      3348
weighted avg       0.98      0.98      0.98      3348

- Zbiór testowy:
	ACCURACY: 77.6371%
              precision    recall  f1-score   support

  ActionName       0.65      0.72      0.69        80
NoActionName       0.85      0.80      0.83       157

   micro avg       0.78      0.78      0.78       237
   macro avg       0.75      0.76      0.76       237
weighted avg       0.78      0.78      0.78       237


>>> Capability:

# RELATIONS
Vectorizer: W słowniku znajduje się 16276 różnych słów
- Zbiór treningowy:
	ACCURACY: 98.1481%
              precision    recall  f1-score   support

  Ca

In [7]:
'''
Grid Search here, for one of the classes
- still not very satisfactory ...
'''

from sklearn.model_selection import GridSearchCV

parameters = [
    {'kernel': ['rbf', 'poly' ,'sigmoid'], 'gamma': [1e-3, 1e-4, 'auto'],
        'C': [10, 50, 100, 200, 300]},
    {'kernel': ['linear'], 'C': [10, 50, 100, 200, 300]}
]

scores = ['roc_auc', 'f1', 'precision', 'recall']

vectorizer = CountVectorizer(
                 preprocessor=dummy, 
                 ngram_range=(1, 1),
                 analyzer='word'
             )

train = dataframes['ActionName']['Train']
test = dataframes['ActionName']['Test']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score.upper())
    print()

    clf = GridSearchCV(SVC(), parameters, cv=5,
#                        scoring='%s_macro' % score
                       scoring=score
                      )
    
    pipeline = Pipeline([      
        ('vectorizer', vectorizer),
        ('clf', clf ),
    ])

    pipeline.fit(train['text-rel-tokens'], train['label_num'])
    clf = pipeline.named_steps['clf']

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = test['label_num'], pipeline.predict(test['text-rel-tokens'])
    print(classification_report(y_true, y_pred))
    print()


# Tuning hyper-parameters for ROC_AUC

Best parameters set found on development set:

{'C': 50, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.734 (+/-0.119) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.660 (+/-0.095) for {'C': 10, 'gamma': 0.001, 'kernel': 'poly'}
0.719 (+/-0.124) for {'C': 10, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.716 (+/-0.119) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.658 (+/-0.088) for {'C': 10, 'gamma': 0.0001, 'kernel': 'poly'}
0.716 (+/-0.119) for {'C': 10, 'gamma': 0.0001, 'kernel': 'sigmoid'}
0.717 (+/-0.121) for {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.659 (+/-0.096) for {'C': 10, 'gamma': 'auto', 'kernel': 'poly'}
0.717 (+/-0.120) for {'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}
0.759 (+/-0.110) for {'C': 50, 'gamma': 0.001, 'kernel': 'rbf'}
0.659 (+/-0.093) for {'C': 50, 'gamma': 0.001, 'kernel': 'poly'}
0.753 (+/-0.123) for {'C': 50, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.719 (+/-0.124) for {'C': 50, 'gamma': 0.0

/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells1

/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells1

/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells1

Best parameters set found on development set:

{'C': 10, 'kernel': 'linear'}

Grid scores on development set:

0.278 (+/-0.178) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.000 (+/-0.000) for {'C': 10, 'gamma': 0.001, 'kernel': 'poly'}
0.082 (+/-0.071) for {'C': 10, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.000 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.000 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'poly'}
0.000 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'sigmoid'}
0.028 (+/-0.050) for {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.000 (+/-0.000) for {'C': 10, 'gamma': 'auto', 'kernel': 'poly'}
0.002 (+/-0.007) for {'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}
0.535 (+/-0.191) for {'C': 50, 'gamma': 0.001, 'kernel': 'rbf'}
0.000 (+/-0.000) for {'C': 50, 'gamma': 0.001, 'kernel': 'poly'}
0.466 (+/-0.186) for {'C': 50, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.082 (+/-0.071) for {'C': 50, 'gamma': 0.0001, 'kernel': 'rbf'}
0.000 (+/-0.000) for {'C': 50,

/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/hom

/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/hom

/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/jells123/anaconda3/envs/py36/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/hom

Best parameters set found on development set:

{'C': 50, 'gamma': 'auto', 'kernel': 'rbf'}

Grid scores on development set:

0.655 (+/-0.263) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.000 (+/-0.000) for {'C': 10, 'gamma': 0.001, 'kernel': 'poly'}
0.636 (+/-0.450) for {'C': 10, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.000 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.000 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'poly'}
0.000 (+/-0.000) for {'C': 10, 'gamma': 0.0001, 'kernel': 'sigmoid'}
0.497 (+/-0.643) for {'C': 10, 'gamma': 'auto', 'kernel': 'rbf'}
0.000 (+/-0.000) for {'C': 10, 'gamma': 'auto', 'kernel': 'poly'}
0.200 (+/-0.800) for {'C': 10, 'gamma': 'auto', 'kernel': 'sigmoid'}
0.662 (+/-0.125) for {'C': 50, 'gamma': 0.001, 'kernel': 'rbf'}
0.000 (+/-0.000) for {'C': 50, 'gamma': 0.001, 'kernel': 'poly'}
0.668 (+/-0.148) for {'C': 50, 'gamma': 0.001, 'kernel': 'sigmoid'}
0.636 (+/-0.450) for {'C': 50, 'gamma': 0.0001, 'kernel': 'rbf'}
0.000 (+/-0.000)

In [37]:
import sklearn
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'brier_score_loss',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'mutual_info_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'v_measure_score']

In [ ]:
Next step:
    Try ONE vs ONE SVC classifier!

In [4]:
import data_helpers
import os
import pandas as pd

dataframes = {
    'All' : {}
}

path = os.getcwd() + "/Dataframes/All/Processed/"
ls = os.listdir(path)
print(ls)

for file in ls:
    if '.csv' in file and 'old' not in file:
        
        print(path + file)
        df = pd.read_csv(path + file, encoding='utf-8')
        
        df.rename({"Unnamed: 0":"a"}, axis="columns", inplace=True)
        df.drop(["a"], axis=1, inplace=True)

        df.rename({"Unnamed: 0.1":"a"}, axis="columns", inplace=True)
        df.drop(["a"], axis=1, inplace=True)
        
        df_type = file.split('-')[1]
        
        dataframes['All'][df_type] = df

['All-Train-P.csv', '.ipynb_checkpoints', 'All-Test-P.csv', 'All-Test-P_old.csv', 'All-Train-P_old.csv']
/home/jells123/Documents/ENGINEER/Secure-NLP/Dataframes/All/Processed/All-Train-P.csv
/home/jells123/Documents/ENGINEER/Secure-NLP/Dataframes/All/Processed/All-Test-P.csv


In [5]:
train[['text-rel-tokens', 'ActionName_num', 'Capability_num']]

NameError: name 'train' is not defined

In [7]:
from clf_helpers import train_and_test

train = dataframes['All']['Train']
test = dataframes['All']['Test']

data_c = 'text-rel-tokens'
label_c = 'Capability_num'

pipeline = Pipeline([      
    ('vectorizer',
     CountVectorizer(
         preprocessor=dummy, 
         ngram_range=(1, 3),
         analyzer='word',
         binary=False,
#          max_features=2000
     )),
    ('clf', SVC(
        decision_function_shape='ovo',
        gamma=1e-1,
        C=10 ** 8,
        kernel='linear'
    )),
])

train_and_test(pipeline, train, test, data_c, label_c, accuracy = True, top_features = False, report = True)

- Zbiór treningowy:
	ACCURACY: 99.5221%
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       531
           1       1.00      0.99      0.99       580
           2       1.00      0.99      1.00       525
           3       0.99      1.00      1.00      1712

   micro avg       1.00      1.00      1.00      3348
   macro avg       1.00      0.99      0.99      3348
weighted avg       1.00      1.00      1.00      3348

- Zbiór testowy:
	ACCURACY: 63.6943%
              precision    recall  f1-score   support

           0       0.45      0.42      0.44        90
           1       0.57      0.73      0.64        62
           2       0.40      0.77      0.53        26
           3       0.76      0.67      0.72       293

   micro avg       0.64      0.64      0.64       471
   macro avg       0.55      0.65      0.58       471
weighted avg       0.66      0.64      0.64       471



In [34]:
GridSearchCV??

In [ ]:
import warnings
warnings.filterwarnings('ignore')

'''
Grid Search REVISITED
for 'ovo' decision function shape -
taking all classes into consideration at once
'''

from sklearn.model_selection import GridSearchCV

parameters = [
    {'kernel': ['rbf', 'poly' ,'sigmoid'], 'gamma': [1e-4, 1e-8, 1e-12, 1e-16, 'auto'],
        'C': [10 ** k for k in [7, 8, 9, 10]]},
    {'kernel': ['linear'], 'C': [10 ** k for k in [7, 8, 9, 10]]}
]

scores = ['f1_weighted', 'accuracy']

vectorizer = CountVectorizer(
                 preprocessor=dummy, 
                 ngram_range=(1, 1),
                 analyzer='word'
             )

for score in scores:
    print("# Tuning hyper-parameters for %s" % score.upper())
    print()

    clf = GridSearchCV(
        SVC(decision_function_shape='ovo'), 
        parameters, cv=5,
        scoring=score
    )
    
    pipeline = Pipeline([      
        ('vectorizer', vectorizer),
        ('clf', clf ),
    ])

    pipeline.fit(train[data_c], train[label_c])
    clf = pipeline.named_steps['clf']

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = test[label_c], pipeline.predict(test[data_c])
    print(classification_report(y_true, y_pred))
    print()


# Tuning hyper-parameters for F1_WEIGHTED

